# Рекомендация тарифов

**Описание проекта**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

**План проекта:**
- Импортировать библиотеки, открыть и изучить датасет
- Разбить файл на выборки для последующего обучения и проверки результатов;
- Обучить модели на разных алгоритмах, выбрать лучшие алгоритмы и проверить на валидационных данных;
- Проверить лучшие модели на тестовых данных, определить лучшую;
- Проверить модели на адеквантость с помощью метрик precision и recall.


## Откройте и изучите файл

In [1]:
#Импортируем библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
#Читаем файл
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
#Изучаем общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
#Знакомимся с данными
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## Разбейте данные на выборки

In [5]:
#Добавляем фичи и таргет
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Теперь разделим данные на 3 части в пропорции 60/20/20, где 60% — обучающая выборка, 20% — валидационная выборка и 20% — тестовая выборка.

In [6]:
#Делим данные на 60% и 40%
features_train, features_to_split, target_train, target_to_split = train_test_split(
    features, target, test_size=0.4, random_state=12345)

#Делим полученные 40% пополам
features_valid, features_test, target_valid, target_test = train_test_split(
    features_to_split, target_to_split, test_size=0.5, random_state=12345)

In [7]:
#Проверяем корректность разделения
print(features_train.shape)
print(features_test.shape)
print(features_valid.shape)
print(target_train.shape)
print(target_test.shape)
print(target_valid.shape)

(1928, 4)
(643, 4)
(643, 4)
(1928,)
(643,)
(643,)


Выборки разделены корректно. Можем приступать к обучению моделей и выявлению оптимальных гиперпараметров.

## Исследуйте модели

Обучим модель по алгоритму дерева решений и найдем оптимальное соотношение гиперпараметров.

In [8]:
#Алгоритм дерева решений
best_tree_criterion = None
best_tree_depth = None
best_tree_result = 0
best_tree_model = None

#Определяем оптимальную глубину дерева
for criterion in ['gini', 'entropy']:
    for depth in range(1, 11):
        model = DecisionTreeClassifier(
            random_state=12345,
            criterion=criterion,
            max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions)
        if result > best_tree_result:
            best_tree_criterion = criterion
            best_tree_depth = depth
            best_tree_result = result
            best_tree_model = model
            
print(f'Лучший критерий: {best_tree_criterion}')        
print(f'Лучшая глубина дерева решений: {best_tree_depth}')
print(f'Лучшая accuracy дерева решений: {best_tree_result:.3}')

Лучший критерий: gini
Лучшая глубина дерева решений: 3
Лучшая accuracy дерева решений: 0.785


Обучим модель по алгоритму случайного леса и найдем оптимальное соотношение гиперпараметров.

In [9]:
#Алгоритм случайного леса
best_forest_depth = None
best_forest_est = None
best_forest_result = 0
best_forest_model = None

#Определяем оптимальную глубину дерева и их количество
for depth in range(1, 11):
    for est in range(1, 101, 10):
        model = RandomForestClassifier(
            random_state=12345,
            n_estimators=est,
            max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions)
        if result > best_forest_result:
            best_forest_depth = depth
            best_forest_est = est
            best_forest_result = result
            best_forest_model = model

print(f'Лучшая глубина дерева: {best_forest_depth}')            
print(f'Лучшее количество деревьев в лесу: {best_forest_est}')
print(f'Лучшая accuracy случайного леса: {best_forest_result:.3}')

Лучшая глубина дерева: 8
Лучшее количество деревьев в лесу: 41
Лучшая accuracy случайного леса: 0.807


Обучим модель по алгоритму логистической регрессии и найдем оптимальное соотношение гиперпараметров.

In [10]:
#Алгоритм логистической регрессии
best_regression_iter = None
best_regression_result = 0
model = None

#Определяем оптимальное число итераций
for iter in range(100, 10000, 100):
    model = LogisticRegression(
        random_state=12345,
        solver='lbfgs',
        max_iter=iter)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_regression_result:
        best_regression_iter = iter
        best_regression_result = result
        best_regression_model = model
        
print(f'Лучшее число итераций: {best_regression_iter}')
print(f'Лучшая accuracy логистической регрессии: {best_regression_result:.3}')

Лучшее число итераций: 100
Лучшая accuracy логистической регрессии: 0.711


- Можем сделать вывод, что, как и ожидалось, лучший результат показывает модель случайного леса с глубиной дерева 8 и количеством деревьев в лесу 41. Показатель accuracy модели случайного леса составляет 0.807, что выше требуемого в условии значения в 0.75.
Тем не менее, эта модель обучается гораздо дольше остальных.

- Модель дерева решений также показывает неплохой результат — при глубине дерева в 3 показатель accuracy составляет 0.785 и также подходит по условию.
Обучение модели идет довольно быстро.

- Худший результат у модели логистической регрессии — при 100 итерациях показатель accuracy составляет 0.71 и не подходит по условию.

Однако финальный вывод делать рано, нужно еще проверить модели на тестовой выборке.

## Проверьте модель на тестовой выборке

In [11]:
#Алгоритм дерева решений
tree_test_predict = best_tree_model.predict(features_test)
tree_accuracy_score = accuracy_score(target_test, tree_test_predict)

print(f'Точность классификации модели дерева решений: {tree_accuracy_score:.3}')

Точность классификации модели дерева решений: 0.779


In [12]:
#Алгоритм случайного леса
best_forest_model
forest_test_predict = best_forest_model.predict(features_test)
forest_accuracy_score = accuracy_score(target_test, forest_test_predict)

print(f'Точность классификации модели случайного леса: {forest_accuracy_score:.3}')

Точность классификации модели случайного леса: 0.798


In [13]:
#Логистическая регрессия
best_regression_model
regression_test_predict = best_regression_model.predict(features_test)
regression_accuracy_score = accuracy_score(target_test, regression_test_predict)

print(f'Точность классификации модели логистической регрессии: {regression_accuracy_score:.3}')

Точность классификации модели логистической регрессии: 0.684


После проверки лучших моделей на тестовой выборке лидерство не изменилось:
- Модель случайного леса показывает accuracy 0.798 и получает первое место, а также подходит по условию задачи.
- Модель дерева решений показывает accuracy 0.779, занимает второе место и подходит по условию задачи.
- Модель логистической регрессии показывает худший результат с accuracy 0.684 и не подходит по условию задачи.

## Проверка модели на адекватность

Проверим лучшие модели по дополнительным метрикам precision и recall.

Precision говорит о том, какая доля положительных ответов предсказана правильно.

Recall говорит о доле правильных ответов среди всех положительных.

In [14]:
tree_precision_score = precision_score(target_test, tree_test_predict)
tree_recall_score = recall_score(target_test, tree_test_predict)

print(f'Оценка точности (precision) модели дерева решений: {tree_precision_score:.3}')
print(f'Оценка полноты (recall) модели дерева решений: {tree_recall_score:.3}')

Оценка точности (precision) модели дерева решений: 0.744
Оценка полноты (recall) модели дерева решений: 0.458


In [15]:
forest_precision_score = precision_score(target_test, forest_test_predict)
forest_recall_score = recall_score(target_test, forest_test_predict)

print(f'Оценка точности (precision) модели случайного леса: {forest_precision_score:.3}')
print(f'Оценка полноты (recall) модели случайного леса: {forest_recall_score:.3}')

Оценка точности (precision) модели случайного леса: 0.752
Оценка полноты (recall) модели случайного леса: 0.537


In [16]:
regression_precision_score = precision_score(target_test, regression_test_predict)
regression_recall_score = recall_score(target_test, regression_test_predict)

print(f'Оценка точности (precision) модели логистической регрессии: {regression_precision_score:.3}')
print(f'Оценка полноты (recall) модели логистической регрессии: {regression_recall_score:.2}')

Оценка точности (precision) модели логистической регрессии: 0.5
Оценка полноты (recall) модели логистической регрессии: 0.039


- Модель дерева решений: precision 0.744 и recall 0.458 говорят о том, что модель получилась среднего качества, а в части recall даже ниже среднего.
- Модель случайного леса: precision 0.752 и recall 0.537 также показывают, что модель среднего качества, но все же лучше модели дерева решений.
- Модель логистической регрессии: precision 0.5 и recall 0.0394 говорит об отвратительном качестве модели, лишь половина положительных ответов предсказана правильно, а доля правильных ответов среди всех положительных составляет ниже 4%.

И снова лидером становится модель случайного леса.

**Итоги:**

Мы обучили модели по каждому из трех алгоритмов классификации, нашли лучшие модели для каждого из алгоритмов и проверили качество их предсказаний.

1. Первое место по всем пунктам исследования занимает модель по алгоритму случайного леса:
- Accuracy на тестовой выборке: 0.798
- Precision на тестовой выборке: 0.752
- Recall на тестовой выборке: 0.537

2. Второе место занимает модель по алгоритму дерева решений:
- Accuracy на тестовой выборке: 0.779
- Precision на тестовой выборке: 0.744
- Recall на тестовой выборке: 0.458

3. Третье место пзанимает модель по алгоритму логистической регрессии:
- Accuracy на тестовой выборке: 0.684
- Precision на тестовой выборке: 0.5
- Recall на тестовой выборке: 0.039

По условию задачи, по параметру accuracy в 0.75 проходят только модели на 1 и 2 месте, однако лучшая модель — модель случайного леса, её и стоит использовать.

Стоит добавить, что в обучающем датасете не так много строк для обучения. Чем выше объем данных, тем точнее будут предсказания моделей, правда обучаться они будут дольше, но ведь это не так важно там, где важна точность.